## Implementación de los métodos

En este _notebook_ se plantea la solución del problema utilizando los algoritmos planteados. Este archivo está autocontenido, sin embargo, la implementación principal se realiza con enfoque _modular_.

**Nota:** Esta implementación se basa en material y actividades impartidas por los profesores de los cursos de [Métodos Numéricos y optimización](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/IV.optimizacion_convexa_y_machine_learning/4.3.Regresion_logistica_R.ipynb) (2010-I) (Prof. Erick Palacios Moreno) y Aprendizaje de Máquina (2019-II) (Prof. Rodrigo Mendoza Smith).


A continuación se describe el planteamiento del problemna, y su implementación con el set de datos utilizado. Una explicación más detallada se realiza en el informe (en formato PDF) de este proyecto.

## Problema de regresión Logística

Inicialmente se considera un conjunto de datos que incorpora una variable output $y_{i}$ asociada a la supervivencia o no del paciente ${i}$ con virus del ébola, y ${j}$ variables explicativas asociadas, $x_{i,j}$. Matemáticamente, este conjunto se define de la siguiente manera: $\mathcal{D}=\left\{ \left(x_{i},y_{i}\right)\in\mathbb{R}^{p}\times\left\{ 0,1\right\} :i\in\left[m\right]\right\} $.

El método de _regresión logística_ asume que $Pr\left[y_{i}\mid x_{i},\beta\right]\sim Bernoulli\left(\mu_{i}\right)$
con los siguientes supuestos sobre la media, $\mu_{i}$:

$$
\mu_{i}=\sigma\left(\beta^{T}x_{i}\right) \label{eq-3.1} \tag{1}
$$
$$
\sigma(z)=\left(1+\exp\left(-z\right)\right)^{-1} \label{eq-3.2} \tag{2}
$$

donde $\beta\in\mathbb{R}^{p}$. 


Dado lo anterior, nuestro problema es encontrar un modelo tal que $\hat{\beta}\in\mathbb{R}^{p}$ explica de la mejor manera posible a $\mathcal{D}$. 

Para lograr lo anterior, debemos estimar el conjunto de parámetros $\hat{\beta}$ para modelar $Pr\left[y\mid x,\hat{\beta}\right]$ y predecir la etiqueta $\hat{y}\in\left\{ 0,1\right\} $ de un nuevo
dato $x$ por medio de:

$$
\hat{y}=\begin{cases}
1 & si\,\sigma\left(\hat{\beta}^{T}x\right)\geq0.5\\
0 & si\,\sigma\left(\hat{\beta}^{T}x\right)<0.5
\end{cases}\label{eq-3.3} \tag{3}
$$

la función de pérdida que queremos minimizar en este problema corresponde a la _log-verosimilitud negativa_ , que está dada por:

$$
F(\beta):=LVN(\beta)=-\sum_{i=1}^{m}\left[y_{i}log\mu_{i}+(1-y_{i})log(1-\mu_{i})\right]\label{eq-3.4} \tag{4}
$$


Una vez planteado lo anterior, queremos encontrar $\hat{\beta}$ por medio de métodos numéricos de optimización de tal forma que se minimize ([4](#mjx-eqn-eq1)) para el conjunto de datos dado.


_En los siguientes fragmentos de código se realiza el planteamiento del problema, desde la importación de datos hasta el proceso de entrenamiento del modelo utilizando distintos algoritmos de optimización que se explican con brevedad._


---------------------

In [1]:
# librerías
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

### Importación de datos

En esta sección se importa y transforma los datos, con el fin de obtener el conjunto $\mathcal{D}$.

In [ ]:
url="https://raw.githubusercontent.com/afcarl/ebola-imc-public/master/data/kenema/test/pres-kgh/imputation-50.csv"
df_raw=pd.read_csv(url,sep=",")

In [ ]:
df_raw.head()
# df[df.isnull().any(axis=1)] - no hay NAs

In [ ]:
# verificar tipo de variables 
df_raw.dtypes

In [ ]:
# resumen de las variables
df_raw.describe()

**Algunas notas sobre los datos**

- _Variables numéricas_ :
    - `CT`
    - `TEMP`
    
- _Variables categóricas_ :
    - `AGE`: se establecen 3 grupos de edades utilizando el percetil 25 (22 años), percentil 50 (36 años) y percentil 75 (45 años)
    - `HEADCH`: binaria
    - `BLEED`: binaria
    - `DIARR`: binaria
    - `VOMIT`: binaria
    - `PABD`: binaria
    - `WEAK`: binaria
    
   
- Para este conjunto de datos la variable `JAUN` no tiene variabilidad, por lo tanto no es una variable, y se omite.

Dado lo anterior, se ajusta el set de datos.

In [ ]:
# ajustes en df_raw 
df_proc = df_raw
df_proc['INTER_AGE'] = "NA"

df_proc.dtypes

In [ ]:
# ajustes en df_raw 
df_proc = df_raw

# para la variable edad se crean cuatro categorías
age_p25 = math.ceil(df_proc['AGE'].quantile(.25))
age_p50 = math.ceil(df_proc['AGE'].quantile(.50))
age_p75 = math.ceil(df_proc['AGE'].quantile(.75))

df_proc['INTER_AGE'] = "NA"
df_proc.loc[(df_proc['AGE'] <= age_p25), 'INTER_AGE'] = 1
df_proc.loc[(df_proc['AGE'] > age_p25) & (df_proc['AGE'] <= age_p50), 'INTER_AGE'] = 2
df_proc.loc[(df_proc['AGE'] > age_p50) & (df_proc['AGE'] <= age_p75), 'INTER_AGE'] = 3
df_proc.loc[(df_proc['AGE'] > age_p75), 'INTER_AGE'] = 4

## one hot encoding
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(df_proc[['INTER_AGE']]).toarray())
enc_df = enc_df.rename(columns={0: f"hasta{age_p25}", 1: f"entre{age_p25+1}y{age_p50}", 2: f"entre{age_p50+1}y{age_p75}", 3:f"mayor{age_p75}"})
# merge with main df bridge_df on key values
df_proc = df_proc.join(enc_df)

# se asignan como categoricas a las binarias, incluido el output
bin_vars = ['OUT', 'HEADCH', 'BLEED', 'DIARR', 'JAUN', 'VOMIT',
       'PABD', 'WEAK', 'INTER_AGE', f"hasta{age_p25}", f"entre{age_p25+1}y{age_p50}", f"entre{age_p50+1}y{age_p75}", f"mayor{age_p75}"]
for var in bin_vars:
    df_proc[var] = df_proc[var].astype('category')
    
# se omiten las variables JAUN, AGE, INTER_AGE
del_vars = ["JAUN", "AGE", "INTER_AGE"]
for var in del_vars:
    df_proc = df_proc.drop(var, axis=1)    
    
# se comprueban los tipos de variable
df_proc.dtypes

In [ ]:
df_proc

### Planteamiento del problema de regresión

A continuación se plantea el código que computa las ecuaciones ([1](#mjx-eqn-eq1)), ([2](#mjx-eqn-eq1)) y ([4](#mjx-eqn-eq1)), planteadas inicialmente.

In [2]:
def sigmoide(z):
    '''
    Devuelve el sigmoide de un vector
        ** Parámetros:
            - z (vec): vector numérico de m entradas
        ** Salidas
            - (float64) valor entre -1 y 1
    '''
    return 1/(1+np.exp(-z).astype(float))
    
def calc_mu(X,beta):
    '''
    Calcula la media para una variable aleatoria con distribución bernoulli.
        ** Parámetros:
            - X (mat): matriz de mxp entradas
            - beta (vec): vector de p entradas
        ** Salidas
            - mu (vec): vector de m entradas
    '''
    a = np.matmul(beta,np.transpose(X))
    mu = sigmoide(a)

    return mu
    
def f(X,y,beta):
    '''
    Función que computa la log-verosimilitud negativa
    ** Parámetros:
        - X (mat): matriz de mxp entradas
        - y (vec): vector de de m entradas de la variable output
        - beta (vec): vector de p entradas
    ** Salidas
        - lvn (int): log-verosimilitud negativa
    '''
    prob = calc_mu(X,beta)
    # log-verosimilitud negativa 
    lvn = -sum(y*np.log(prob)+(1-y)*(np.log(1-prob)))
    return lvn

Reescribiendo la ecuación de la función de pérdida ([4](#mjx-eqn-eq1)), tenemos:

$$F(\beta)=- \sum_{i=1}^{m}[y_i log\mu_i + (1-y_i)log(1-\mu_i)]$$

Las expresiones correspondientes al gradiente y a la matriz hessiana asociados a este problema, se plantean a continuación:

\begin{align}
\nabla F(\beta) & =\frac{d}{d\beta}F(\beta)\nonumber \\
 & =\sum_{i}\left(\mu_{i}-y_{i}\right)x_{i}\nonumber \\
 & =\boldsymbol{X}^{T}\left(\boldsymbol{\mu}-\boldsymbol{y}\right)\label{eq:gradient}
\end{align}

Por otro lado, la ecuación que describe la matrix Hessiana es la siguiente:

\begin{align}
\nabla^{2}F(\beta) & =\frac{d}{d\beta}\nabla F\left(\beta\right)^{T}\nonumber \\
 & =\sum_{i}\left(\nabla_{\beta}\mu_{i}\right)x_{i}^{T}\nonumber \\
 & =\sum_{i}\mu_{i}\left(1-\mu_{i}\right)x_{i}x_{i}^{T}\nonumber \\
 & =\boldsymbol{X^{T}SX}\label{eq:hessian}
\end{align}

donde $\boldsymbol{S}\triangleq diag\left(\mu_{i}\left(1-\mu_{i}\right)\right)$.
Como es resaltado por Murphy (2012), es definida positiva, lo que implica que ([4](#mjx-eqn-eq1)) es convexa
y tiene un mínimo global.

In [3]:
def gradiente_f(X,y,beta):
    '''
    Calcula el gradiente asociado la log-verosimilitud negativa del problema de regresión logística
        ** Parámetros:
            - X (mat): matriz de mxp entradas
            - y (vec): vector de de m entradas de la variable output
            - beta (vec): vector de p entradas
        ** Salidas
            - grad (vec): vector de m entradas
    '''
    mu=calc_mu(X,beta)
    grad = np.matmul(np.transpose(X), mu-y)
    return grad

def hessiana_f(X,y,beta):
    '''
    Calcula la matriz Hessiana asociada a la log-verosimilitud negativa del problema de regresión logística
        ** Parámetros:
            - X (mat): matriz de mxp entradas
            - y (vec): vector de de m entradas de la variable output
            - beta (vec): vector de p entradas
        ** Salidas
            - grad (vec): vector de m entradas
    '''
    mu=calc_mu(X,beta)
    S=np.diag(mu*(1-mu))
    hes=np.matmul(np.transpose(X),np.matmul(S,X))
    return hes

In [4]:
def normalize(x):
    return x/np.sqrt(sum(x*x))

In [5]:
def clasifica(X, beta_hat):
    mu=calc_mu(X,beta_hat)
    yhat=mu
    yhat[mu<.5]=0
    yhat[mu>=.5]=1
    return yhat

In [6]:
def descent_direction(X, y, beta, method="max"):
    '''
    This function determines the direction of the descent pk=-inv(Bk) grad f
    '''
    if(method=="max"):
        pk=gradiente_f(X,y,beta)
    
    elif(method=="newton"):
        grad=gradiente_f(X,y,beta)
        hess=hessiana_f(X,y,beta)
        inv_hess=np.linalg.inv(hess)
        pk=np.matmul(inv_hess,grad)
    
    
    return normalize(pk)


In [7]:
def gradient_descent(X, y,lr=.1, tol=10**(-6), max_iter=10**5, method="max"):
    '''
    Devuelve vector de parámetros beta (px1) resultante del proceso de optimización por descenos de gradiente
        ** Parámetros:
            - X (mat): matriz de mxp entradas
            - y (vec): vector de de m entradas de la variable output
            - lr (float64): tasa de aprendizaje
            - tol (float64): criterio de convergencia
            - max_iter (int): número máximo de iteraciones
            - method (str): método que determina la dirección de descenso
                opciones:
                    - max: método de descenso
                    - newton: método de Newton
                    - 
            
        ** Salidas
            - beta_new (vec): vector de p entradas con parámetros que minimizan la función de pérdida
    '''
    
    #inicializa
    iteraciones=0
    
    #inicializamos beta aleatoria
    beta=np.random.uniform(0,1,X.shape[1])
    
    #primera iteracion
    pk = descent_direction(X, y, beta, method)
    beta_new= beta - lr*pk
    
    #condición de paro.
    #El cambio total es menor que la tolerancia
    while ((abs(f(X,y,beta) - f(X,y,beta_new))>tol) & (iteraciones<max_iter)):
        iteraciones+=1 #contador de ciclo
        
        beta = beta_new
        pk = descent_direction(X,y,beta,method)
        beta_new = beta - lr*pk

        #if iteraciones>max_iter:
        #    break
    
    print("iteraciones=",iteraciones)
    return beta_new

**Nota:** Implementación tomando datos anteriores - se debe corregir la matriz de diseño

In [8]:
url="https://raw.githubusercontent.com/afcarl/ebola-imc-public/master/data/kenema/test/pres-kgh/imputation-50.csv"
df=pd.read_csv(url,sep=",")

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

data =df.to_numpy()
y = data[:,0]
X = data[:,1:]
x_train, x_test, y_train, y_test=train_test_split(X,y,test_size=.2)

#scale data
scaler=MinMaxScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.fit_transform(x_test)

In [13]:
beta_hat=gradient_descent(x_train,y_train)
yhat=clasifica(x_test,beta_hat)

print("beta_hat=", beta_hat)
print("Error de clasificacion=",round(100*sum(abs(y_test-yhat))/len(yhat),2),"%")

iteraciones= 644
beta_hat= [-9.27894027  5.41544927 24.74342114 -6.53123606  0.2179865   2.8421769
  0.4173048  10.0601313   5.32053725  4.00423041]
Error de clasificacion= 22.73 %


In [ ]:
# Método de Newton (no está funcionando)
beta_hat=gradient_descent(x_train,y_train, method="newton")
yhat=clasifica(x_test,beta_hat)

print("beta_hat=", beta_hat)
print("Error de clasificacion=",round(100*sum(abs(y_test-yhat))/len(yhat),2),"%")